# RAG using Meta AI Llama-3


<img src="./resources/rag_architecture.png" width=800px>

In [3]:
#install modules
%pip install python-dotenv  --quiet
%pip install llama-index --quiet
%pip install llama-index-llms-ollama --quiet
%pip install llama-index-embeddings-huggingface --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [54]:
pip install llama-index-postprocessor-cohere_rerank --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.40.0 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader

In [5]:
# allows nested access to the event loop
nest_asyncio.apply()

In [6]:
# add your documents in this directory, you can drag & drop
input_dir_path = '/home/ec2-user/SageMaker/RAG Llama3/test-dir'

In [7]:

# setup llm & embedding model
llm=Ollama(model="llama3", request_timeout=120.0)
# embed_model = HuggingFaceEmbedding( model_name="Snowflake/snowflake-arctic-embed-m", trust_remote_code=True)
embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)

In [8]:
# load data
loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".pdf"],
            recursive=True
        )
docs = loader.load_data()

# Creating an index over loaded data
Settings.embed_model = embed_model
index = VectorStoreIndex.from_documents(docs, show_progress=True)



Parsing nodes:   0%|          | 0/11 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/13 [00:00<?, ?it/s]

In [9]:
# Create the query engine, where we use a cohere reranker on the fetched nodes
Settings.llm = llm
query_engine = index.as_query_engine()


In [10]:
# ====== Customise prompt template ======
qa_prompt_tmpl_str = (
"Context information is below.\n"
"---------------------\n"
"{context_str}\n"
"---------------------\n"
"Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
"Query: {query_str}\n"
"Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)



In [11]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)


In [12]:
# Generate the response
response = query_engine.query("What is the summary of this paper?",)

ConnectError: [Errno 111] Connection refused

In [6]:
display(Markdown(str(response)))

Based on the provided context, DSPy stands for "Declarative Programming for Self-Refining Language Model Pipelines". It's a framework that allows developers to programmatically solve advanced tasks with language and retrieval models through composing and declaring modules.